In [142]:
import numpy as np
import pandas as pd
from datetime import datetime,date,time
from dateutil.relativedelta import relativedelta


### **A) Coleta dos Dados de Direção do Vento**

In [143]:
source_file = 'raw/BAUET2_CAETITÉ_2_PRE_Verif_EPE.txt'

In [144]:
df_pressao = pd.read_csv(source_file, delimiter = ';' , header=None )

### **B) Sobre os Dados no Patio 1**

#### <span style="color:#DC143C">**B1.Formato dos Dados**

In [145]:
df_pressao.shape # formato da matriz

(1622, 49)

#### <span style="color:#DC143C">**B2.Visao do Dataframe**

In [146]:
df_pressao.head(2)  # visão de 5 linhas

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48
0,20170101,999.0,999.000,882.333,882.333,882.0,882.0,882.000,882.000,882.333,883.0,883.0,883.000,883.0,884.0,884.0,884.0,885.0,885.0,885.000,885.0,885.0,885.0,885.0,885.000,885.0,884.667,884.000,884.0,883.667,883.333,883.0,883.0,883.0,883.0,883.0,883.0,883.0,883.0,883.0,883.0,883.0,883.0,884.0,884.000,884.0,884.000,883.333,883.0
1,20170102,883.0,882.667,882.000,882.000,882.0,882.0,881.667,881.333,882.000,882.0,882.0,881.667,882.0,882.0,882.0,883.0,883.0,883.0,883.667,884.0,884.0,884.0,884.0,883.667,883.0,883.000,882.333,882.0,882.000,882.000,881.0,881.0,881.0,881.0,881.0,881.0,881.0,881.0,881.0,881.0,881.0,881.0,881.0,881.333,881.0,881.667,882.000,882.0


### **C) Extração os Dados de 2017**

#### <span style="color:#DC143C"> **C1. Converter a primeira coluna em Data**

In [147]:
df_pressao[0]= pd.to_datetime(df_pressao[0], format='%Y%m%d')

#### <span style="color:#DC143C"> **C2.Remove registros fora do Intervalo**

In [148]:
df_pressao = df_pressao.drop(df_pressao[df_pressao[0] > '2017-12-31'].index)
df_pressao = df_pressao.drop(df_pressao[df_pressao[0] < '2017-01-01'].index)

In [149]:
df_pressao['longitude'] = -42.75
df_pressao['latitude']  = -14.25 

In [150]:
cols = list(df_pressao)
cols = [cols[-1]] + cols[:-1]
df_pressao = df_pressao[cols]

cols = list(df_pressao)
cols = [cols[-1]] + cols[:-1]
df_pressao = df_pressao[cols]

In [151]:
df_pressao = df_pressao.reset_index()

### **D) Formatação dos Dados**

#### <span style="color:#DC143C"> **D1. Migrar os dados para o Formato**

In [152]:
frente = ['Longitude', 'Latitude', 'Data', 'Valor30_1', 'Valor30_2']  # Cabeçalho
df_result_pressao = pd.DataFrame(columns=frente)  # Criação de um dataframe vazio
    
Linha1    =  df_pressao.shape[0] 
Colunas1  =  df_pressao.shape[1] 

for lin2 in range(Linha1):  
    refhora = 0
    
    for col in range (Colunas1):        
        if col > 0:            
        
            if col % 2 != 0: # Se o número (col) for impar                   
               
                if col < 49:
                    
                    Data = df_pressao.loc[lin2,0]
                                  
                    hora = time(hour=refhora, minute=0, second=0)            
                    dataCompleta = datetime.combine(Data, hora)
                    #print(dataCompleta)
                    lin_alvo = [df_pressao.loc[lin2,'longitude'], df_pressao.loc[lin2,'latitude'], dataCompleta , df_pressao.loc[lin2,col], df_pressao.loc[lin2,col+1]] 
                    # print(lin_alvo)
                    df_result_pressao.loc[len(df_result_pressao)] = lin_alvo  # adding a row
                         
                refhora = refhora + 1          
                


#### <span style="color:#DC143C"> **D2. Informações sobre os Tipos de Dados**

In [153]:
df_result_pressao.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8760 entries, 0 to 8759
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Longitude  8760 non-null   float64       
 1   Latitude   8760 non-null   float64       
 2   Data       8760 non-null   datetime64[ns]
 3   Valor30_1  8760 non-null   float64       
 4   Valor30_2  8760 non-null   float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 410.6 KB


In [154]:
df_result_pressao.shape

(8760, 5)

#### <span style="color:#DC143C"> **D3. Remove ruido dos Dados**

In [155]:
df_result_pressao.loc[df_result_pressao['Valor30_1']== 999.00,'Valor30_1'] = 0
df_result_pressao.loc[df_result_pressao['Valor30_1']== 99.000,'Valor30_1'] = 0

df_result_pressao.loc[df_result_pressao['Valor30_2']== 99.000,'Valor30_2'] = 0
df_result_pressao.loc[df_result_pressao['Valor30_2']== 999.00,'Valor30_2'] = 0

save_file = 'd:\df_result_pressao.csv'
df_result_pressao.to_csv(save_file, index = True)

colMean1 = df_result_pressao['Valor30_1'].sum() / ((df_result_pressao['Valor30_1'] != 0).sum())
colMean1 = round(colMean1, 2)
print('Media 1:' + str(colMean1))

colMean2 = df_result_pressao['Valor30_2'].sum() / ((df_result_pressao['Valor30_2'] != 0).sum())
colMean2 = round(colMean2, 2)
print('Media 2:' + str(colMean2))

In [156]:
#df_result_pressao.loc[df_result_pressao['Valor30_1']== 0,'Valor30_1'] = 884.49
#df_result_pressao.loc[df_result_pressao['Valor30_2']== 0,'Valor30_2'] = 884.43

#df_result_pressao.loc[df_result_pressao['Valor30_1']< 600,'Valor30_1'] = 884.49
#df_result_pressao.loc[df_result_pressao['Valor30_2']< 600,'Valor30_2'] = 884.43

In [157]:
# Removendo as linhas sem valores
df_result_pressao = df_result_pressao.drop(df_result_pressao[df_result_pressao['Valor30_1'] == 0].index)
df_result_pressao = df_result_pressao.drop(df_result_pressao[df_result_pressao['Valor30_2'] == 0].index)
df_result_pressao = df_result_pressao.drop(df_result_pressao[df_result_pressao['Valor30_1'] < 600].index)
df_result_pressao = df_result_pressao.drop(df_result_pressao[df_result_pressao['Valor30_2'] < 600].index)

#### <span style="color:#DC143C"> **D3. Agregar Magnitude para 1H**

In [158]:
df_result_pressao

,Longitude,Latitude,Data,Valor30_1,Valor30_2
1,-42.75,-14.25,2017-01-01 01:00:00,882.333,882.333
2,-42.75,-14.25,2017-01-01 02:00:00,882.000,882.000
3,-42.75,-14.25,2017-01-01 03:00:00,882.000,882.000
4,-42.75,-14.25,2017-01-01 04:00:00,882.333,883.000
5,-42.75,-14.25,2017-01-01 05:00:00,883.000,883.000
...,...,...,...,...,...
8754,-42.75,-14.25,2017-12-31 18:00:00,882.000,882.000
8755,-42.75,-14.25,2017-12-31 19:00:00,882.000,883.000
8756,-42.75,-14.25,2017-12-31 20:00:00,883.000,883.667
8757,-42.75,-14.25,2017-12-31 21:00:00,884.000,884.000


In [159]:
df_result_pressao['pressao_Verif'] = (df_result_pressao['Valor30_1'] + df_result_pressao['Valor30_2']) / 2

In [160]:
# Formatar o resultado com 2 casas decimais
df_result_pressao['pressao_Verif'] = df_result_pressao['pressao_Verif'].round(decimals=2)

In [161]:
df_result_pressao = df_result_pressao.drop('Valor30_1', 1)
df_result_pressao = df_result_pressao.drop('Valor30_2', 1)

In [162]:
df_result_pressao.head(10)

,Longitude,Latitude,Data,pressao_Verif
1,-42.75,-14.25,2017-01-01 01:00:00,882.33
2,-42.75,-14.25,2017-01-01 02:00:00,882.00
3,-42.75,-14.25,2017-01-01 03:00:00,882.00
4,-42.75,-14.25,2017-01-01 04:00:00,882.67
5,-42.75,-14.25,2017-01-01 05:00:00,883.00
6,-42.75,-14.25,2017-01-01 06:00:00,883.50
7,-42.75,-14.25,2017-01-01 07:00:00,884.00
8,-42.75,-14.25,2017-01-01 08:00:00,885.00
9,-42.75,-14.25,2017-01-01 09:00:00,885.00
10,-42.75,-14.25,2017-01-01 10:00:00,885.00


### **E. Exporta Resultados**

In [163]:
output_file = 'processed/teste_03_patio_pressao.csv'
df_result_pressao.to_csv(output_file, sep=';', encoding='utf-8')